# Load Dataset

In [ ]:
# set cuda device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


In [25]:
import datasets 
from datasets import load_dataset

dataset = load_dataset("PolyAI/banking77")

Generating test split: 100%|██████████| 3080/3080 [00:00<00:00, 21173.15 examples/s]


In [26]:
len(dataset['train']), len(dataset['test'])

(10003, 3080)

In [28]:
from utils import *
import llama
llama_config = ModelArguments()
llama_config.model = "meta-llama/Llama-2-7b-chat-hf"
llama_config.temperature = 0.3
llama_config.max_new_tokens = 100
llama_config.top_k = 5
llama_model = llama.LlamaModel(llama_config)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.52it/s]
/home/lynie/miniconda3/envs/eff/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Llama Model loaded


In [58]:
from transformers.models.llama.tokenization_llama import B_SYS, E_SYS
SystemPrompt = '''You are an expert assistant in the field of customer service. Your task is to help workers in the customer service department of a company. 
Your task is to classify the customer's question in order to help the customer service worker to answer the question. In order to help the worker you MUST respond with the number and the name of one of the following classes you know. 
In case you reply with something else, you will be penalized.'''
SystemPrompt += ''' The classes are: \n
- activate_my_card
- age_limit
- apple_pay_or_google_pay
- atm_support
- automatic_top_up
- balance_not_updated_after_bank_transfer
- balance_not_updated_after_cheque_or_cash_deposit
- beneficiary_not_allowed
- cancel_transfer
- card_about_to_expire
- card_acceptance
- card_arrival
- card_delivery_estimate
- card_linking
- card_not_working
- card_payment_fee_charged
- card_payment_not_recognised
- card_payment_wrong_exchange_rate
- card_swallowed
- cash_withdrawal_charge
- cash_withdrawal_not_recognised
- change_pin
- compromised_card
- contactless_not_working
- country_support
- declined_card_payment
- declined_cash_withdrawal
- declined_transfer
- direct_debit_payment_not_recognised
- disposable_card_limits
- edit_personal_details
- exchange_charge
- exchange_rate
- exchange_via_app
- extra_charge_on_statement
- failed_transfer
- fiat_currency_support
- get_disposable_virtual_card
- get_physical_card
- getting_spare_card
- getting_virtual_card
- lost_or_stolen_card
- lost_or_stolen_phone
- order_physical_card
- passcode_forgotten
- pending_card_payment
- pending_cash_withdrawal
- pending_top_up
- pending_transfer
- pin_blocked
- receiving_money
- Refund_not_showing_up
- request_refund
- reverted_card_payment
- supported_cards_and_currencies
- terminate_account
- top_up_by_bank_transfer_charge
- top_up_by_card_charge
- top_up_by_cash_or_cheque
- top_up_failed
- top_up_limits
- top_up_reverted
- topping_up_by_card
- transaction_charged_twice
- transfer_fee_charged
- transfer_into_account
- transfer_not_received_by_recipient
- transfer_timing
- unable_to_verify_identity
- verify_my_identity
- verify_source_of_funds
- verify_top_up
- virtual_card_not_working
- visa_or_mastercard
- why_verify_identity
- wrong_amount_of_cash_received
- wrong_exchange_rate_for_cash_withdrawal
'''
Prompt = '''Here is the user query: {} \n If multiple classes match the query intention, respond with the MOST precise class name.'''
PROMPT = " ".join(["[INST]", B_SYS, SystemPrompt, E_SYS, Prompt, "[/INST]"])

In [67]:
id_to_label = {0: "activate_my_card",
1: "age_limit",
2: "apple_pay_or_google_pay",
3: "atm_support",
4: "automatic_top_up",
5: "balance_not_updated_after_bank_transfer",
6: "balance_not_updated_after_cheque_or_cash_deposit",
7: "beneficiary_not_allowed",
8: "cancel_transfer",
9: "card_about_to_expire",
10: "card_acceptance",
11: "card_arrival",
12: "card_delivery_estimate",
13: "card_linking",
14: "card_not_working",
15: "card_payment_fee_charged",
16: "card_payment_not_recognised",
17: "card_payment_wrong_exchange_rate",
18: "card_swallowed",
19: "cash_withdrawal_charge",
20: "cash_withdrawal_not_recognised",
21: "change_pin",
22: "compromised_card",
23: "contactless_not_working",
24: "country_support",
25: "declined_card_payment",
26: "declined_cash_withdrawal",
27: "declined_transfer",
28: "direct_debit_payment_not_recognised",
29: "disposable_card_limits",
30: "edit_personal_details",
31: "exchange_charge",
32: "exchange_rate",
33: "exchange_via_app",
34: "extra_charge_on_statement",
35: "failed_transfer",
36: "fiat_currency_support",
37: "get_disposable_virtual_card",
38: "get_physical_card",
39: "getting_spare_card",
40: "getting_virtual_card",
41: "lost_or_stolen_card",
42: "lost_or_stolen_phone",
43: "order_physical_card",
44: "passcode_forgotten",
45: "pending_card_payment",
46: "pending_cash_withdrawal",
47: "pending_top_up",
48: "pending_transfer",
49: "pin_blocked",
50: "receiving_money",
51: "Refund_not_showing_up",
52: "request_refund",
53: "reverted_card_payment?",
54: "supported_cards_and_currencies",
55: "terminate_account",
56: "top_up_by_bank_transfer_charge",
57: "top_up_by_card_charge",
58: "top_up_by_cash_or_cheque",
59: "top_up_failed",
60: "top_up_limits",
61: "top_up_reverted",
62: "topping_up_by_card",
63: "transaction_charged_twice",
64: "transfer_fee_charged",
65: "transfer_into_account",
66: "transfer_not_received_by_recipient",
67: "transfer_timing",
68: "unable_to_verify_identity",
69: "verify_my_identity",
70: "verify_source_of_funds",
71: "verify_top_up",
72: "virtual_card_not_working",
73: "visa_or_mastercard",
74: "why_verify_identity",
75: "wrong_amount_of_cash_received",
76: "wrong_exchange_rate_for_cash_withdrawal"}

In [78]:
from tqdm import tqdm
correct = 0

for i in tqdm(range(50)):
    pred = llama_model.predict(PROMPT.format(dataset['train'][i]['text']))
    gold = id_to_label[dataset['train'][i]['label']]
    if gold in pred[0]:
        correct += 1
correct/50

100%|██████████| 50/50 [02:11<00:00,  2.63s/it]


0.44

In [62]:
dataset['train'][10000]['label']

24

In [27]:
# check the label distribution of sampled dataset
import pandas as pd
pd.Series(dataset['train']['label']).value_counts()

15    187
28    182
6     181
75    180
19    177
     ... 
41     82
18     61
10     59
72     41
23     35
Name: count, Length: 77, dtype: int64

In [6]:
# save the sampled dataset
sampled_dataset.save_to_disk('data/10000_sampled_agnews')

Saving the dataset (1/1 shards): 100%|██████████| 10000/10000 [00:00<00:00, 194239.19 examples/s]
